In [ ]:
!pip -q install -U transformers datasets accelerate evaluate scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 117.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 91.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [ ]:
import torch, json, os, random
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving 혐오탐지데이터 2만.jsonl to 혐오탐지데이터 2만 (3).jsonl


In [ ]:
import json

DATA_PATH = list(uploaded.keys())[0]

def load_jsonl(path):
    rows = []
    with open(path, "r", encoding="utf-8-sig") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            rows.append(json.loads(line))
    print(f"✅ 로드 완료: {len(rows)} rows")
    return rows

rows = load_jsonl(DATA_PATH)
print(rows[0])

✅ 로드 완료: 20022 rows
{'utterance': ['A: 오늘 지각할 뻔했다ㅋㅋ 버스 진짜 안 옴', 'B: 나도 거의 뛰어서 옴;;', 'A: 수학쌤 오늘 또 지옥 시간일 듯', 'B: 그 ㅆㅂ 늙은이 목소리만 들어도 현타 옴'], 'target_index': 3, 'label': 'toxic'}


In [ ]:
# === 모든 자음/모음 반복 제거 ===
import re

# 한글 자모(ㄱ~ㅎ, ㅏ~ㅣ) 범위
RE_JAMO_REPEAT = re.compile(r"([ㄱ-ㅎㅏ-ㅣ])\1{1,}")  # 같은 자모가 2번 이상 반복될 때

def strip_repeats(text: str) -> str:
    if not isinstance(text, str):
        return text
    # 자모 반복 제거
    text = RE_JAMO_REPEAT.sub(r"\1", text)
    # 공백 정리
    return re.sub(r"\s{2,}", " ", text).strip()

In [ ]:
# === rows 정규화 → [CTX]/[TGT] text 생성 → train/valid Dataset ===
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split

def build_text_from_ctx(ctx_list, target_index):
    ctx = "\n".join(strip_repeats(u) for u in ctx_list[:target_index]) if target_index > 0 else ""
    tgt = strip_repeats(ctx_list[target_index])
    return f"[CTX]\n{ctx}\n[/CTX]\n[TGT] {tgt} [/TGT]".strip()

def label_to_int(v):
    s = str(v).strip().lower()
    return 1 if s in {"1","true","toxic","yes","y"} else 0

# rows -> norm(list of dicts with text,label)
norm = []
for r in rows:
    if "utterance" in r and "target_index" in r and "label" in r:
        txt = build_text_from_ctx(r["utterance"], int(r["target_index"]))
        norm.append({"text": txt, "label": label_to_int(r["label"])})
    elif "text" in r and "label" in r:
        norm.append({"text": strip_repeats(r["text"]), "label": label_to_int(r["label"])})
    else:
        raise ValueError(f"지원하지 않는 형식: {r.keys()}")

print("✅ 정규화 샘플:", norm[0])

# train/valid split (90:10, stratify)
df = pd.DataFrame(norm)
tr, va = train_test_split(df, test_size=0.1, random_state=42, stratify=df["label"])

train = Dataset.from_pandas(tr.reset_index(drop=True))
valid = Dataset.from_pandas(va.reset_index(drop=True))

print("✅ train size:", len(train), "| valid size:", len(valid))
print("✅ train[0]:", train[0])

✅ 정규화 샘플: {'text': '[CTX]\nA: 오늘 지각할 뻔했다 버스 진짜 안 옴\nB: 나도 거의 뛰어서 옴;;\nA: 수학쌤 오늘 또 지옥 시간일 듯\n[/CTX]\n[TGT] B: 그 ㅆㅂ 늙은이 목소리만 들어도 현타 옴 [/TGT]', 'label': 1}
✅ train size: 18019 | valid size: 2003
✅ train[0]: {'text': '[CTX]\nA: 애니 굿즈 예약했어. 한정판이야.\nB: 빠르네.\nA: 아니면 금방 품절돼.\n[/CTX]\n[TGT] B: 나는 굳이 그런 거에 돈 쓰는 건 이해 안 가긴 해. [/TGT]', 'label': 1}


In [ ]:
# === 토크나이저/모델 준비 + 토크나이즈 ===
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

MODEL_NAME = "monologg/kobert"  # 원하면 다른 베이스로 교체 가능
MAX_LEN = 384

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

def tok(batch):
    return tokenizer(batch["text"], truncation=True, padding=False, max_length=MAX_LEN)

train_tok = train.map(tok, batched=True)
valid_tok = valid.map(tok, batched=True)

keep_cols = ["input_ids", "attention_mask", "label"]
train_tok = train_tok.remove_columns([c for c in train_tok.column_names if c not in keep_cols]).with_format("torch")
valid_tok = valid_tok.remove_columns([c for c in valid_tok.column_names if c not in keep_cols]).with_format("torch")

print("✅ tokenized train sample shapes:",
      train_tok[0]["input_ids"].shape, train_tok[0]["attention_mask"].shape)


tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

The repository monologg/kobert contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/monologg/kobert .
 You can inspect the repository content at https://hf.co/monologg/kobert.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] ㅛ
The repository monologg/kobert contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/monologg/kobert .
 You can inspect the repository content at https://hf.co/monologg/kobert.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


tokenization_kobert.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/monologg/kobert:
- tokenization_kobert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_78b3253a26.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/18019 [00:00<?, ? examples/s]

Map:   0%|          | 0/2003 [00:00<?, ? examples/s]

✅ tokenized train sample shapes: torch.Size([76]) torch.Size([76])


In [ ]:
# === PATCH: FocalTrainer 시그니처 + processing_class 사용 ===
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# 클래스 가중치 (이전 계산 재사용; 없으면 여기서 다시 계산)
try:
    class_weights
except NameError:
    labels_np = np.array([int(x) for x in train["label"]])
    pos_ratio = labels_np.mean() if len(labels_np) else 0.5
    w_pos = max(1.0, (1.0 - pos_ratio + 1e-6) / (pos_ratio + 1e-6))
    w_neg = 1.0
    class_weights = torch.tensor([w_neg, w_pos], dtype=torch.float)

class SmoothedFocalLoss(nn.Module):
    def __init__(self, gamma=2.0, label_smoothing=0.05, class_weights=None, reduction='mean'):
        super().__init__()
        self.gamma = gamma
        self.eps = label_smoothing
        self.class_weights = class_weights
        self.reduction = reduction

    def forward(self, logits, target):
        num_classes = logits.size(-1)
        with torch.no_grad():
            true_dist = torch.zeros_like(logits)
            true_dist.fill_(self.eps / (num_classes - 1))
            true_dist.scatter_(1, target.unsqueeze(1), 1.0 - self.eps)

        log_probs = F.log_softmax(logits, dim=-1)
        probs = log_probs.exp()
        ce = -(true_dist * log_probs).sum(dim=-1)
        pt = (true_dist * probs).sum(dim=-1).clamp_min(1e-8)
        focal = (1 - pt) ** self.gamma
        loss = focal * ce

        if self.class_weights is not None:
            w = self.class_weights.to(logits.device).gather(0, target)
            loss = loss * w

        return loss.mean() if self.reduction == 'mean' else loss.sum()

class FocalTrainer(Trainer):
    def __init__(self, *args, gamma=2.0, label_smoothing=0.05, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.crit = SmoothedFocalLoss(gamma=gamma, label_smoothing=label_smoothing,
                                      class_weights=class_weights)

    # ✅ 최신 시그니처 반영 (num_items_in_batch 추가)
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.get("labels")
        outputs = model(**{k: v for k, v in inputs.items() if k != "labels"})
        logits = outputs.logits
        loss = self.crit(logits, labels)
        return (loss, outputs) if return_outputs else loss

def compute_metrics(eval_pred):
    logits, labels_np = eval_pred
    probs = (np.exp(logits) / np.exp(logits).sum(-1, keepdims=True))[:, 1]
    preds = (probs >= 0.5).astype(int)
    p, r, f1, _ = precision_recall_fscore_support(labels_np, preds, average="binary", zero_division=0)
    acc = accuracy_score(labels_np, preds)
    return {"accuracy": acc, "precision": p, "recall": r, "f1": f1}

args = TrainingArguments(
    output_dir="/content/toxic_ctx_retrain_focal",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    fp16=torch.cuda.is_available(),
    logging_steps=50,
    report_to="none",
    seed=42,
    gradient_accumulation_steps=2,
    warmup_ratio=0.06,
    lr_scheduler_type="linear",
    max_grad_norm=1.0,
)

trainer = FocalTrainer(
    model=model,
    args=args,
    train_dataset=train_tok,
    eval_dataset=valid_tok,
    processing_class=tokenizer,   # ✅ tokenizer 대신
    compute_metrics=compute_metrics,
    gamma=2.0,
    label_smoothing=0.05,
    class_weights=class_weights,
)

trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=2))
import types, os

assert hasattr(tokenizer, "save_vocabulary"), "tokenizer가 아직 안 만들어졌어요."

_orig_save_vocab = tokenizer.save_vocabulary

def _save_vocabulary_patched(self, save_directory, filename_prefix=None):
    # KoBert는 filename_prefix 인자를 지원 안 하므로 무시하고 원본 호출
    return _orig_save_vocab(save_directory)

tokenizer.save_vocabulary = types.MethodType(_save_vocabulary_patched, tokenizer)
print("✅ Patched: KoBertTokenizer.save_vocabulary now ignores filename_prefix")
trainer.train()
print("✅ 학습 완료 (Focal+LS+ES)")


✅ Patched: KoBertTokenizer.save_vocabulary now ignores filename_prefix


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.002500,0.043908,0.984024,0.981132,0.987013,0.984064
2,0.009500,0.030865,0.983025,0.985930,0.980020,0.982966
3,0.001300,0.043692,0.982027,0.987867,0.976024,0.981910


✅ 학습 완료 (Focal+LS+ES)


In [ ]:
from google.colab import drive

# 1) 드라이브 마운트
drive.mount('/content/drive')

# 2) 저장 경로 지정 (원하는 폴더로 변경 가능)
SAVE_DIR = "/content/drive/MyDrive/혐오탐지모델 마지막시도"

# 3) 모델 저장
trainer.save_model(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)

print(f"✅ 모델 저장 완료 → {SAVE_DIR}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ 모델 저장 완료 → /content/drive/MyDrive/혐오탐지모델 마지막시도


In [ ]:
# === 검증셋으로 Temperature(T) & Threshold 최적화 ===
import torch, numpy as np, torch.nn as nn
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix

model.eval()
device = model.device if hasattr(model, "device") else ("cuda" if torch.cuda.is_available() else "cpu")

# 1) 검증셋 로짓/라벨 뽑기 (Trainer 활용)
pred_out = trainer.predict(valid_tok)  # logits: np.ndarray [N,2], label_ids: [N]
logits_np = pred_out.predictions
labels_np = pred_out.label_ids.astype(int)

# 안전 체크
assert logits_np.ndim == 2 and logits_np.shape[1] == 2, f"logits shape 이상: {logits_np.shape}"

logits = torch.from_numpy(logits_np).to(torch.float32)
labels = torch.from_numpy(labels_np)

# 2) Temperature 학습 (NLL 최소화)
T = torch.tensor(1.0, requires_grad=True)
opt = torch.optim.LBFGS([T], lr=0.1, max_iter=50, line_search_fn="strong_wolfe")
ce = nn.CrossEntropyLoss()

def closure():
    opt.zero_grad()
    loss = ce(logits / T, labels)
    loss.backward()
    return loss

opt.step(closure)
best_T = float(T.detach().cpu().item())
print(f"✅ Best Temperature T = {best_T:.6f}")

# 3) 최적 Threshold 스윕 (F1 최대)
probs = torch.softmax(logits / best_T, dim=-1)[:, 1].cpu().numpy()  # toxic=class 1
ths = np.linspace(0.01, 0.99, 99)

best = {"f1": -1, "thr": None, "p": None, "r": None, "cm": None}
for thr in ths:
    pred = (probs >= thr).astype(int)
    p, r, f1, _ = precision_recall_fscore_support(labels_np, pred, average="binary", zero_division=0)
    tn, fp, fn, tp = confusion_matrix(labels_np, pred).ravel()
    if f1 > best["f1"]:
        best.update({"f1": f1, "thr": float(thr), "p": p, "r": r, "cm": {"tn":int(tn),"fp":int(fp),"fn":int(fn),"tp":int(tp)}})

print(f"✅ Best Threshold = {best['thr']:.6f}")
print(f"   F1={best['f1']:.4f}, P={best['p']:.4f}, R={best['r']:.4f}, CM={best['cm']}")

# 4) 상수로 바로 적용(원하면 저장 없이 고정)
TEMP   = best_T
THRESH = best["thr"]
print(f"🔧 Using TEMP={TEMP:.6f}, THRESH={THRESH:.6f}")

✅ Best Temperature T = 1.055155
✅ Best Threshold = 0.450000
   F1=0.9841, P=0.9802, R=0.9880, CM={'tn': 982, 'fp': 20, 'fn': 12, 'tp': 989}
🔧 Using TEMP=1.055155, THRESH=0.450000


In [ ]:
# === Inference (No rules) + Context Gate + Diagnostics (grad×emb) ===
# - 사용: predict_last_utterance_from_console_with_diag() 실행 후 A/B 한줄씩 입력 → 빈 줄 종료
# - 또는 포맷/리스트로 바로: build_text_from_utterances([...]) → predict_ctx_with_diagnostics([...])

import re, torch, numpy as np
import torch.nn.functional as F

# ---- Config (너 결과 반영) ----
TEMP       = 1.055155   # calibrated T
THRESH     = 0.45       # calibrated threshold
MAX_LEN    = 384
GATE_DELTA = 0.30       # |p_ctx - p_tgt| >= 0.30 이면 TGT-only 사용
TOPK       = 8          # 진단용 top-k 토큰

# ---- Helpers ----
_TGT_RE = re.compile(r"\[TGT\]\s*(.*?)\s*\[/TGT\]", re.DOTALL)

def build_text_from_utterances(utterances, target_index=None):
    """utterances: ['A: ...', 'B: ...', ...]; 마지막 줄을 TGT로"""
    if not utterances:
        raise ValueError("최소 1줄 이상 입력해 주세요.")
    if target_index is None:
        target_index = len(utterances) - 1
    ctx = "\n".join(utterances[:target_index]) if target_index > 0 else ""
    tgt = utterances[target_index]
    return f"[CTX]\n{ctx}\n[/CTX]\n[TGT] {tgt} [/TGT]".strip(), tgt

def _extract_tgt(formatted: str) -> str:
    m = _TGT_RE.search(formatted)
    return m.group(1).strip() if m else formatted.strip()

def _prob_toxic(texts):
    """toxic(1) 확률만 반환 (TS 적용)"""
    device = model.device if hasattr(model, 'device') else ('cuda' if torch.cuda.is_available() else 'cpu')
    enc = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=MAX_LEN)
    enc = {k: v.to(device) for k, v in enc.items()}
    with torch.no_grad():
        logits = model(**enc).logits / float(TEMP)
        p = F.softmax(logits, dim=-1)[:, 1].cpu().numpy()
    return p

# ---- Diagnostics (grad × emb) ----
def _token_importance_one(text, target_class=1, topk=TOPK):
    """단일 텍스트에 대해 grad×emb 토큰 기여도 산출 + 편향 플래그"""
    device = model.device if hasattr(model, 'device') else ('cuda' if torch.cuda.is_available() else 'cpu')
    model.eval()

    enc = tokenizer(text, return_tensors="pt", truncation=True, padding=False, max_length=MAX_LEN)
    enc = {k: v.to(device) for k, v in enc.items()}

    emb_layer = model.get_input_embeddings()
    cache = {}
    def fwd_hook(module, inp, out):
        cache["emb"] = out
        out.retain_grad()
    h = emb_layer.register_forward_hook(fwd_hook)

    logits = model(**enc).logits / float(TEMP)
    prob = F.softmax(logits, dim=-1)[0, target_class]
    model.zero_grad(set_to_none=True)
    logits[0, target_class].backward()

    emb  = cache["emb"][0].detach()       # [seq, hid]
    grad = cache["emb"].grad[0].detach()  # [seq, hid]
    scores = (emb * grad).sum(dim=-1)     # [seq]
    scores_np = scores.cpu().numpy()

    input_ids = enc["input_ids"][0].cpu().tolist()
    toks = tokenizer.convert_ids_to_tokens(input_ids)

    # special 토큰/괄호/자모 영향도 집계
    spec = set(getattr(tokenizer, "all_special_tokens", []))
    def is_bracket(t):
        return t in {"[", "]", "▁[", "▁]", "/"} or t.startswith("[/") or t.startswith("▁[/")
    def is_jamo(t):
        return any(('\u3131' <= ch <= '\u318E') or ('\u1100' <= ch <= '\u11FF') for ch in t)

    total_pos = float(sum(max(0.0, float(s)) for s in scores_np)) or 1.0
    br_pos = jamo_pos = spec_pos = 0.0
    keep_tok, keep_score = [], []
    for tk, sc in zip(toks, scores_np):
        if tk not in spec:
            keep_tok.append(tk); keep_score.append(sc)
        if sc > 0:
            if is_bracket(tk): br_pos  += float(sc)
            if is_jamo(tk):    jamo_pos+= float(sc)
            if tk in spec:     spec_pos+= float(sc)

    # 상위 양/음수 토큰만 뽑기 (special 제외)
    pos_pairs = sorted([(t, float(s)) for t, s in zip(keep_tok, keep_score) if s > 0],
                       key=lambda x: x[1], reverse=True)[:topk]
    neg_pairs = sorted([(t, float(s)) for t, s in zip(keep_tok, keep_score) if s < 0],
                       key=lambda x: x[1])[:topk]

    flags = {
        "bracket_bias_pct": 100.0 * br_pos / total_pos,
        "jamo_bias_pct":    100.0 * jamo_pos / total_pos,
        "special_bias_pct": 100.0 * spec_pos / total_pos,
    }

    h.remove()
    return {
        "prob": float(prob.item()),
        "top_pos": pos_pairs,
        "top_neg": neg_pairs,
        "flags": flags,
        "tokens": toks,              # 필요시 참고용
        "scores": scores_np.tolist() # 필요시 참고용
    }

# ---- Main Predictors ----
def predict_ctx_with_diagnostics(formatted_texts, gate_delta=GATE_DELTA, topk=TOPK):
    """formatted_texts: ['[CTX]..[/CTX]\\n[TGT]..[/TGT]', ...]"""
    p_ctx = _prob_toxic(formatted_texts)
    tgts = [_extract_tgt(t) for t in formatted_texts]
    tgt_only = [f"[CTX]\n\n[/CTX]\n[TGT] {t} [/TGT]" for t in tgts]
    p_tgt = _prob_toxic(tgt_only)

    out = []
    for full, pc, pt, tgt in zip(formatted_texts, p_ctx, p_tgt, tgts):
        use_tgt = abs(pc - pt) >= gate_delta
        pu = pt if use_tgt else pc
        pred = 1 if pu >= THRESH else 0
        reason = "tgt_only_override" if use_tgt else "ctx_used"

        diag_ctx = _token_importance_one(full,  target_class=1, topk=topk)
        diag_tgt = _token_importance_one(f"[CTX]\n\n[/CTX]\n[TGT] {tgt} [/TGT]", target_class=1, topk=topk)

        out.append({
            "tgt": tgt,
            "prob_ctx": float(pc),
            "prob_tgt": float(pt),
            "prob_used": float(pu),
            "threshold": THRESH,
            "pred_label": int(pred),      # 1 toxic / 0 non-toxic
            "reason": reason,
            "top_pos_ctx": diag_ctx["top_pos"],
            "top_neg_ctx": diag_ctx["top_neg"],
            "top_pos_tgt": diag_tgt["top_pos"],
            "top_neg_tgt": diag_tgt["top_neg"],
            "flags_ctx": diag_ctx["flags"],  # {'bracket_bias_pct':.., 'jamo_bias_pct':.., 'special_bias_pct':..}
            "flags_tgt": diag_tgt["flags"],
        })
    return out

def predict_last_utterance_from_console_with_diag(gate_delta=GATE_DELTA, topk=TOPK):
    print("A/B 한 줄씩 입력하세요. (빈 줄로 종료)")
    lines = []
    while True:
        try:
            line = input().rstrip("\n")
        except EOFError:
            break
        if not line.strip():
            break
        lines.append(line.strip())

    formatted, _ = build_text_from_utterances(lines, target_index=None)
    res = predict_ctx_with_diagnostics([formatted], gate_delta=gate_delta, topk=topk)[0]

    label_str = "toxic" if res["pred_label"] == 1 else "non_toxic"
    print("\n=== 결과 ===")
    print(f"TGT: {res['tgt']}")
    print(f"p_ctx={res['prob_ctx']:.4f} | p_tgt={res['prob_tgt']:.4f} | used={res['prob_used']:.4f} (thr={res['threshold']:.3f})")
    print(f"reason={res['reason']} → pred={label_str}")
    print("\n— 토큰 기여도 (CTX+TGT, 독성 ↑ top):", res["top_pos_ctx"])
    print("— 토큰 기여도 (CTX+TGT, 독성 ↓ top):", res["top_neg_ctx"])
    print("— 토큰 기여도 (TGT-only, 독성 ↑ top):", res["top_pos_tgt"])
    print("— 토큰 기여도 (TGT-only, 독성 ↓ top):", res["top_neg_tgt"])
    print("\n— 편향 플래그 (CTX):", res["flags_ctx"])
    print("— 편향 플래그 (TGT):", res["flags_tgt"])
